# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [190]:
import numpy as np
import pandas as pd
import random as rd

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [198]:
df_data = pd.read_csv('..\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('..\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)

# K-means clustering
## als eerste moet de data genormaliseerd worden
de formule hiervoor staat in de casusinformatie. De formule is een bruek, met als teller het coordinaat en de noemer de wortel van de som van de coordinaten in het kwadraat.

In [227]:
def lib(FileName):
    def openFile(FileName):
        read_data = open(f'{FileName}')
        data = read_data.read()
        data = data.splitlines()
        read_data.close()
        
        return data
        
    lib = {}
    
    for lines in openFile(FileName):
        line = lines.split()
        lib[line[0]] = line[1:]
            
    return lib

lib_data = lib('..\Voorbeeld_clusterdata.txt')
lib_results = lib('..\Voorbeeld_clusterresult.txt')

In [228]:
'''het maken van een lijst waar voor elk gen de noemer in komt.'''
noemer = list()
for i in lib_data: 
    noemer.append(sum([float(i)**2 for i in lib_data[i]])**0.5)

In [229]:
'''de lijst van de noemers in een DataFrame zetten, zodat de coordinaten makkelijk gedeeld kunnen worden door de noemer.'''
df_factors = pd.DataFrame(noemer)

In [230]:
'''de coordinaten delen door de noemer en van de array een dataframe maken.'''
df_normalised = pd.DataFrame(np.divide(df_data.values, df_factors.values), )
df_normalised.columns = df_data.columns

## nu geven we elke vector een random cluster. 

In [231]:
K = 3

In [232]:
def random_clusters(data, K=6):
    '''invoer is dataframe met genormaliseerde data. deze code zorgt ervoor dat alle vectoren in een van de clusters
    worden ingedeeld. De functie wordt net zo vaak doorlopen, totdat elk cluster minimaal 1 gen toegekent heeft gekregen.'''
    random_clusters = []
    
    a = 0
    if a == 0:
        for i in range(len(data)):
            random_clusters.append(rd.randint(1,K))

        df_clusterd_data = data.copy()
        df_clusterd_data['cluster'] = random_clusters
        df_clusterd_data
        
        if range(1,K+1) in random_clusters:
            a = 1
        
    return df_clusterd_data

df_clusterd_data = random_clusters(df_normalised,K)  
df_clusterd_data

,1,2,3,4,5,6,7,8,cluster
0,0.239776,0.203315,0.426405,0.556181,0.386855,0.383765,0.246573,0.228652,1
1,-0.164504,-0.279546,-0.252314,-0.482397,-0.400701,-0.510185,-0.270098,-0.323451,1
2,-0.211364,-0.803356,-0.237999,-0.150361,-0.238859,-0.064440,-0.202772,-0.358288,3
3,0.309072,0.373379,0.392670,0.440096,0.425225,0.347254,0.276919,0.197742,1
4,-0.285344,-0.594142,-0.238438,-0.229057,-0.284563,-0.121174,-0.368993,-0.473750,1
...,...,...,...,...,...,...,...,...,...
1007,0.262366,0.385001,0.331177,0.563782,0.347008,0.275453,0.325267,0.230916,3
1008,0.381695,0.302057,0.256819,0.458505,0.535786,0.244096,0.335043,0.167286,2
1009,-0.488990,-0.747571,-0.066406,0.046283,0.366239,0.177083,0.173058,0.006037,3
1010,0.080280,0.264680,0.329877,0.429618,0.488004,0.315281,0.466109,0.274411,3


In [233]:
def square_dataframe(data):
    '''deze functie neemt het kwadraat van elk getal, zodat later de afstand tussen de vectoren van de 
    genen en die van het centrum van de clusters berekend kunnen worden.
    
    
    now when i am writting these comments I think that I don"t use this function in the rest of the code, but I will
    leave it cause we might need it later'''
    
    clusters = list(data['cluster'])
    data_without_clusters = data.drop(axis=1, labels ='cluster')
    data_squared = (data_without_clusters.copy())**2
    data_squared['cluster'] = clusters
    
    return data_squared
    
df_squared = square_dataframe(data = df_clusterd_data)

In [234]:
def make_vector(data):
    '''this function creates an extra column in the dataframe named "vector". this column 
    contains lists of the vectors from the genes.'''
    vector_list = []

    for index, rows in data.iterrows():
        vector = []
        for coordinate in rows:
            vector.append(coordinate)
        vector_list.append(vector)

    data['vector'] = vector_list
    return data

df_vec = make_vector(data=df_clusterd_data.loc[:, df_clusterd_data.columns != 'cluster'])
clusters = df_clusterd_data['cluster']
df_vec['cluster'] = clusters

In [235]:
def centrum(data):
    '''this function calculates the mean of every coordinate i (i = 1,...,8) for all klusters K
    this function needs a data frame as imput where in the coordinates for all genes are, and the vector of these genes.
    the output of this function is a dataframe with every row corosponding to a cluster. all elements in the row together make the coordinate of the centrum of the cluster
    and the last cell of each row gives a list of the centrum of the vector.'''
    
    df_rest = data['vector'].copy()
    df_use = data.drop(axis=1, labels ='vector')
    df_sum = df_use.groupby('cluster').sum()
    df_centrum = df_sum.rdiv(1) #for every cell, devide 1 by the number in that cell
    
    df_centrum = make_vector(df_centrum)
    
    return df_centrum

df_cluster_centrum = centrum(df_vec) 

In [236]:
def compute_distance(data_genes, data_clusters, K):
    '''This function computes the distance for every gen with all the clusters. The imput that this function need are two data frames
    the first data frame it needs, the dataframe with the vectors for all the genes. the second one is a data frame with the vectors
    for the centra of the clusters.'''
    df_distance = pd.DataFrame(index=data_genes.index)
    centrums = {}
    for vector in range(len(list(data_clusters))):
        centrums[vector] = list(data_clusters)[vector]
    
    for gen in range(len(data_genes)):
        for cluster in range(1, len(data_clusters)+1):
            total = 0
            c = data_clusters[cluster]
            for coordinate in range(len(c)):
                total += (data_genes[gen][coordinate] + c[coordinate])**2
                df_distance.loc[gen,cluster] = (total)**0.5
    
    return df_distance
distance = compute_distance(df_vec['vector'], df_cluster_centrum['vector'],K)

In [237]:
def cluster_smallest_distance(data):
    '''this function looks for every gen, which of the clusters centrum is the least far away and saves this in a list'''
    smallest_distance = data.idxmin(axis=1)
    return smallest_distance

smallest_distance = cluster_smallest_distance(distance)

In [238]:
def check(data, smallest_distance):
    '''this function makes a dataframe with the old cluster and the cluster that is the least far away.'''
    cluster_comparison = data[['cluster']].copy()
    cluster_comparison['new_cluster'] = smallest_distance
    
    
    return cluster_comparison

df_vector = check(df_vec, smallest_distance)

In [239]:
def score(data, df_vec):
    '''in this function compares the two clusters (old and new). for every old and new cluster that are the same (so no
    improvement) the score will decrease with 1. the score starts at the amount of genes there are. a score of 0 means that
    all genes are in the cluster that is the least far away'''
    score = len(data)
    for gen in range(len(data)):
        cluster = data.iloc[gen, 0]
        new_cluster = data.iloc[gen, 1]
        if cluster == new_cluster:
            score -= 1
    df_vec['cluster'] = df_vector['new_cluster']
    return score, df_vec
    
function_output = score(df_vector, df_vec)
score = function_output[0]
df_vector = function_output[1]

In [240]:
def right_cluster(data, score, df_vector, K):
    '''deze functie moet de andere functies doorlopen todat het goede resultaat behaald is. Dat doet ie alleen nog niet 
    helemaal: als de while loop voor een tweede keer doorlopen wordt geeft ie namelijk een foutmelding. dit heeft 
    waarschijnlijk te maken met een tweede aanpassing die we nog moeten doen: als je bijv 13 clusters wilt hebben
    en bij het verbeteren (1e keer while loop) liggen alle genen het dichste bij 3,4 en 8, worden alle genen in die 3
    clusters ingedeeld. Hierdoor krijgt de functie compute_distance niet de juiste invoer waardoor de while loop geen
    tweede keer doorlopen kan worden.'''
    while score > 0:
        df_squared = square_dataframe(data)
        df_cluster_centrum = centrum(df_vector)
        distance = compute_distance(df_vector['vector'], df_cluster_centrum['vector'],K)
        smallest_distance = cluster_smallest_distance(distance)
        df_vector = check(df_vector, smallest_distance)
        score = score(df_vector, df_vec)
        df_vector = score(df_vector, df_vec)[1]
    
    return score

K_means_clusterd = right_cluster(df_clusterd_data,score,df_vector,K)
K_means_clusterd

TypeError: 'int' object is not callable

## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [17]:
class KMCA:
    def __init__(self, k=6, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(K):
        self.k = K

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        def pred(self, data):
            distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            return classification
        
        self.pred_classifications = list()
        
        if np.isfortran(data):
            for element in data:
                self.pred_classifications.append(pred(self, element))
        else:
            self.pred_classifications.append(pred(self, data))
            
        return self.pred_classifications

### Own cluster method:

## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [18]:
kmca = KMCA()
kmca.fit(df_data.values)
df_clusterd['KMCA'] = kmca.predict(df_data.values)

df_clusterd.head()

,KMCA
0,
846160,0
820434,4
849103,2
846353,3
848613,2


### save results:

In [19]:
df_clusterd.to_csv(r'Data\clusterresultaten.txt', header=None, index=True, sep=' ')

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\clusterresultaten.txt'